In [1]:
import numpy as np
import pandas as pd
import sklearn

from scipy import interp
import matplotlib.pyplot as plt
from numpy import ravel

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.model_selection import StratifiedKFold

from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

from sklearn.neural_network import MLPClassifier
import itertools

import sklearn.metrics as metrics

import matplotlib.pyplot as plt

import random

import scipy

from sklearn.preprocessing import StandardScaler  


# define imputer
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
feature_scaler = StandardScaler()  

# Set seed
random.seed(2020)
np.random.seed(2020)

In [2]:
A = pd.read_excel("mortality_all.xlsx")

y = A["Mortality"]
y = y.ravel()

X = A
del X["Mortality"]
del X["ID"]
del X["STS"]
del X["STSACC"]

X10 = X[['pre_hstnt_value', 'MaxLeu', 'Septum', 'MaxCrea', 'LA', 'Fever',
         'Perikarderguss', 'MaxCRP', 'Fluoroscopy.time', 'Contrast.agent.use']]

X = imp.fit_transform(X)
X = pd.DataFrame(data = X) 
X = feature_scaler.fit_transform(X)

X10 = imp.fit_transform(X10)
X10 = pd.DataFrame(data = X10) 
X10 = feature_scaler.fit_transform(X10)


In [3]:
B = pd.read_excel("Baseline August (30 features + ID).xlsx")
B10 = B[['TAPSE', 'RCA_Ao', 'NYHA', 'pre_hstnt_value', 'pre_krea_value',
         'Grad der Verlaklung', 'E_E', 'Sex', 'Minimale Durchmesser', 'PA']]

B = imp.fit_transform(B)
B = pd.DataFrame(data = B) 
B = feature_scaler.fit_transform(B)  

B10 = imp.fit_transform(B10)
B10 = pd.DataFrame(data = B10) 
B10 = feature_scaler.fit_transform(B10)  

In [4]:
inner_cv = KFold(n_splits=4, shuffle=True, random_state=4)
cv       = StratifiedKFold(n_splits = 5, shuffle = True, random_state=4)

In [5]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_jobs=-1,max_features= 'sqrt' ,n_estimators=50, 
                             oob_score = True, class_weight = {1: 18}) 

grid_param_rf = {
    'n_estimators': [200, 700],
    'max_features': ['auto', 'sqrt', 'log2']
}

classifier_rf = GridSearchCV(estimator=rfc, param_grid=grid_param_rf, cv=inner_cv, scoring='roc_auc')

In [6]:
probas_rf    = ravel(np.zeros(len(y)))
probas_rf10  = ravel(np.zeros(len(y)))
probas_rfb   = ravel(np.zeros(len(y)))
probas_rfb10 = ravel(np.zeros(len(y)))

for train, test in cv.split(X, y):
    fit = classifier_rf.fit(X[train], y[train])
    probs = fit.predict_proba(X[test])
    probas_rf[test] = probs[:, 1]
    fit = classifier_rf.fit(X10[train], y[train])
    probs = fit.predict_proba(X10[test])
    probas_rf10[test] = probs[:, 1]
    fit   = classifier_rf.fit(B[train], y[train])
    probs = fit.predict_proba(B[test])
    probas_rfb[test] = probs[:, 1]
    fit   = classifier_rf.fit(B10[train], y[train])
    probs = fit.predict_proba(B10[test])
    probas_rfb10[test] = probs[:, 1]


In [7]:
NeuralNetwork = MLPClassifier(solver='sgd', alpha=1e-5, max_iter = 10000,
                    hidden_layer_sizes=(5, 2), random_state=1)
grid_param_nn = {  
    'activation': ['logistic','tanh','relu'],
    'learning_rate': ['constant','invscaling'],
    'hidden_layer_sizes':[x for x in itertools.product((3,4,5),repeat=3)]
}

classifier_nn = GridSearchCV(estimator=NeuralNetwork, param_grid=grid_param_nn, cv=inner_cv, scoring='roc_auc')

In [8]:
probas_nn    = ravel(np.zeros(len(y)))
probas_nn10  = ravel(np.zeros(len(y)))
probas_nnb   = ravel(np.zeros(len(y)))
probas_nnb10 = ravel(np.zeros(len(y)))

for train, test in cv.split(X, y):
    fit = classifier_nn.fit(X[train], y[train])
    probs = fit.predict_proba(X[test])
    probas_nn[test] = probs[:, 1]
    fit = classifier_nn.fit(X10[train], y[train])
    probs = fit.predict_proba(X10[test])
    probas_nn10[test] = probs[:, 1]
    fit   = classifier_nn.fit(B[train], y[train])
    probs = fit.predict_proba(B[test])
    probas_nnb[test] = probs[:, 1]
    fit   = classifier_nn.fit(B10[train], y[train])
    probs = fit.predict_proba(B10[test])
    probas_nnb10[test] = probs[:, 1]


In [9]:
from sklearn import svm, datasets
svc = svm.SVC(probability = True, class_weight ={1: 18})

grid_param_svm = {'kernel':('linear', 'rbf'), 'C':[1, 10], 'gamma':[.0001, .001, .01]}

classifier_svm = GridSearchCV(estimator=svc, param_grid=grid_param_svm, cv=inner_cv, scoring='roc_auc')

In [10]:
probas_svm    = ravel(np.zeros(len(y)))
probas_svm10  = ravel(np.zeros(len(y)))
probas_svmb   = ravel(np.zeros(len(y)))
probas_svmb10 = ravel(np.zeros(len(y)))

for train, test in cv.split(X, y):
    fit = classifier_svm.fit(X[train], y[train])
    probs = fit.predict_proba(X[test])
    probas_svm[test] = probs[:, 1]
    fit = classifier_svm.fit(X10[train], y[train])
    probs = fit.predict_proba(X10[test])
    probas_svm10[test] = probs[:, 1]
    fit   = classifier_svm.fit(B[train], y[train])
    probs = fit.predict_proba(B[test])
    probas_svmb[test] = probs[:, 1]
    fit   = classifier_svm.fit(B10[train], y[train])
    probs = fit.predict_proba(B10[test])
    probas_svmb10[test] = probs[:, 1]


In [11]:
import scipy.stats

# AUC comparison adapted from
# https://github.com/Netflix/vmaf/
def compute_midrank(x):
    J = np.argsort(x)
    Z = x[J]
    N = len(x)
    T = np.zeros(N, dtype=np.float)
    i = 0
    while i < N:
        j = i
        while j < N and Z[j] == Z[i]:
            j += 1
        T[i:j] = 0.5*(i + j - 1)
        i = j
    T2 = np.empty(N, dtype=np.float)
    # Note(kazeevn) +1 is due to Python using 0-based indexing
    # instead of 1-based in the AUC formula in the paper
    T2[J] = T + 1
    return T2



def fastDeLong(predictions_sorted_transposed, label_1_count, ordered_sample_weight):
    # Short variables are named as they are in the paper
    m = label_1_count
    n = predictions_sorted_transposed.shape[1] - m
    positive_examples = predictions_sorted_transposed[:, :m]
    negative_examples = predictions_sorted_transposed[:, m:]
    k = predictions_sorted_transposed.shape[0]

    tx = np.empty([k, m], dtype=np.float)
    ty = np.empty([k, n], dtype=np.float)
    tz = np.empty([k, m + n], dtype=np.float)
    for r in range(k):
        tx[r, :] = compute_midrank(positive_examples[r, :])
        ty[r, :] = compute_midrank(negative_examples[r, :])
        tz[r, :] = compute_midrank(predictions_sorted_transposed[r, :])
    aucs = tz[:, :m].sum(axis=1) / m / n - float(m + 1.0) / 2.0 / n
    v01 = (tz[:, :m] - tx[:, :]) / n
    v10 = 1.0 - (tz[:, m:] - ty[:, :]) / m
    sx = np.cov(v01)
    sy = np.cov(v10)
    delongcov = sx / m + sy / n
    return aucs, delongcov


def calc_pvalue(aucs, sigma):
    l = np.array([[1, -1]])
    z = np.abs(np.diff(aucs)) / np.sqrt(np.dot(np.dot(l, sigma), l.T))
    return np.log10(2) + scipy.stats.norm.logsf(z, loc=0, scale=1) / np.log(10)


def compute_ground_truth_statistics(ground_truth, sample_weight=None):
    assert np.array_equal(np.unique(ground_truth), [0, 1])
    order = (-ground_truth).argsort()
    label_1_count = int(ground_truth.sum())
    if sample_weight is None:
        ordered_sample_weight = None
    else:
        ordered_sample_weight = sample_weight[order]

    return order, label_1_count, ordered_sample_weight


def delong_roc_variance(ground_truth, predictions):
    sample_weight = None
    order, label_1_count, ordered_sample_weight = compute_ground_truth_statistics(
        ground_truth, sample_weight)
    predictions_sorted_transposed = predictions[np.newaxis, order]
    aucs, delongcov = fastDeLong(predictions_sorted_transposed, label_1_count, ordered_sample_weight)
    assert len(aucs) == 1, "There is a bug in the code, please forward this to the developers"
    return aucs[0], delongcov


def delong_roc_test(ground_truth, predictions_one, predictions_two):
    sample_weight = None
    order, label_1_count, ordered_sample_weight = compute_ground_truth_statistics(ground_truth)
    predictions_sorted_transposed = np.vstack((np.asarray(predictions_one), np.asarray(predictions_two)))[:, order]
    aucs, delongcov = fastDeLong(predictions_sorted_transposed, label_1_count, ordered_sample_weight)
    return 10**calc_pvalue(aucs, delongcov)

P-values comparing all features:

In [12]:
delong_roc_test(y, probas_rf, probas_rf10)

array([[0.08149959]])

In [13]:
delong_roc_test(y, probas_nn, probas_nn10)

array([[0.02711955]])

In [14]:
delong_roc_test(y, probas_svm, probas_svm10)

array([[0.00286767]])

P-values comparing baseline features:

In [15]:
delong_roc_test(y, probas_rfb, probas_rfb10)

array([[0.07968427]])

In [16]:
delong_roc_test(y, probas_nnb, probas_nnb10)

array([[0.03528414]])

In [17]:
delong_roc_test(y, probas_svmb, probas_svmb10)

array([[0.31338372]])